In [3]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.Collecting bs4




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Manju\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the news website
url = "https://www.bbc.com/news"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content, "html.parser")

# Extract headlines
headlines = []
for item in soup.find_all("h3"):
    headline = item.text.strip()
    if headline:  
        headlines.append(headline)

# Save to a DataFrame
data = pd.DataFrame(headlines, columns=["Headline"])

# Save to CSV
data.to_csv("news_headlines.csv", index=False)
print("News headlines collected and saved to 'news_headlines.csv'")



News headlines collected and saved to 'news_headlines.csv'


In [ ]:
# Clean headlines
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text

# Apply preprocessing
data['Cleaned_Headline'] = data['Headline'].apply(preprocess_text)
print("Text preprocessing complete!")


Text preprocessing complete!


In [9]:
pip install azure-ai-textanalytics


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Manju\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Example sample data
data = pd.DataFrame({"Cleaned_Headline": [
    "The product launch was a huge success.",
    "Heavy rains are expected in the coming days.",
    "The team won the championship in a thrilling match.",
    "The stock market experienced a major decline yesterday.",
    "Groundbreaking medical discovery announced today."
]})

# Prepare documents
documents = [{"id": str(i), "text": text} for i, text in enumerate(data['Cleaned_Headline'].tolist())]

print(documents) 


[{'id': '0', 'text': 'The product launch was a huge success.'}, {'id': '1', 'text': 'Heavy rains are expected in the coming days.'}, {'id': '2', 'text': 'The team won the championship in a thrilling match.'}, {'id': '3', 'text': 'The stock market experienced a major decline yesterday.'}, {'id': '4', 'text': 'Groundbreaking medical discovery announced today.'}]


In [12]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Azure credentials (replace with yours)
endpoint = "https://sentimentaltextanalaysis.cognitiveservices.azure.com/"
api_key = "1jwUSTjEc1e0sb3GCYeKlo9bB1y2PC7RFuIg4lwe15xPAMTaMsgyJQQJ99AKACYeBjFXJ3w3AAAEACOGwyYm"

# Authenticate client
def authenticate_client():
    credential = AzureKeyCredential(api_key)
    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=credential)
    return text_analytics_client

client = authenticate_client()

# Prepare data for API
documents = [{"id": str(i), "text": text} for i, text in enumerate(data['Cleaned_Headline'].tolist())]

# Analyze sentiments
def analyze_sentiments(client, documents):
    response = client.analyze_sentiment(documents=documents)
    sentiments = []
    for doc in response:
        if not doc.is_error:
            sentiments.append({
                "Sentiment": doc.sentiment,
                "Positive_Score": doc.confidence_scores.positive,
                "Negative_Score": doc.confidence_scores.negative,
                "Neutral_Score": doc.confidence_scores.neutral
            })
        else:
            sentiments.append({"Sentiment": "Error", "Positive_Score": 0, "Negative_Score": 0, "Neutral_Score": 0})
    return sentiments

# Get sentiment analysis results
sentiment_results = analyze_sentiments(client, documents)

# Add results to DataFrame
for key in sentiment_results[0].keys():
    data[key] = [res[key] for res in sentiment_results]

data.to_csv("sentiment_results.csv", index=False)
print("Sentiment analysis complete! Results saved to 'sentiment_results.csv'")


Sentiment analysis complete! Results saved to 'sentiment_results.csv'


In [14]:
pip install azure-storage-blob


  Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/3.1 MB 30.2 MB/s eta 0:00:00
Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Manju\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
from azure.storage.blob import BlobServiceClient

# Replace these with your values
connection_string = "DefaultEndpointsProtocol=https;AccountName=sentimentalresults;AccountKey=eWDWQjReJKTdBb+EpzAvjtOaMENOJoTlT5eFQX85IrWzwhmiTuK5FXSr0UCRg7CBYNJjGbFuXDxi+AStQky4hA==;EndpointSuffix=core.windows.net"
container_name = "sentimental-results"

# Connect to Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Upload a file
blob_client = blob_service_client.get_blob_client(container=container_name, blob="sentiment_results.csv")

# Upload the file
with open("sentiment_results.csv", "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print("File uploaded to Azure Blob Storage!")


File uploaded to Azure Blob Storage!
